In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.13.1+cu117
Torchvision Version:  0.14.1+cu117


In [ ]:
# Might need to adjust for different data sets
DATA_DIR = "/mnt/d/dataset/imagenet"
# DATA_DIR = "/mnt/d/dataset/stylized-imagenet"

data_dir = DATA_DIR

# May need to adjust per machine specs
BATCH_SIZE = 128 # 128 uses ~14GB, 192 uses 19,5 GB, 256 uses 24GB. 192 ~= 256
INPUT_SIZE = 224
input_size = INPUT_SIZE

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256), # TODO imagenet training uses 256 input size, crop to 224 on the next line
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True, num_workers=4) for x in ['val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [ ]:
def eval_model(model,dataloaders):
    since = time.time()
    model.to('cuda')
    model.eval()

    criterion = nn.CrossEntropyLoss()
    running_loss = 0.0
    running_corrects_top1 = 0
    running_corrects_top5 = 0

    for inputs, labels in dataloaders['val']:


        inputs = inputs.to('cuda')
        labels = labels.to('cuda')

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds_top1 = torch.max(outputs,1)
        _, preds_top5 = torch.topk(outputs,5,1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects_top1 += torch.sum(preds_top1==labels.data)
        running_corrects_top5 += torch.sum(labels.unsqueeze(1).expand_as(preds_top5) == preds_top5).item()

    epoch_loss = running_loss / len(dataloaders['val'].dataset)
    epoch_acc_top1 = (running_corrects_top1.double() / len(dataloaders['val'].dataset)).item()
    epoch_acc_top5 = running_corrects_top5 / len(dataloaders['val'].dataset)
    time_elapsed = time.time() - since
    print(f'{epoch_loss=}, {epoch_acc_top1=}, {epoch_acc_top5=}, {time_elapsed=}')
    return (epoch_loss, epoch_acc_top1, epoch_acc_top5, time_elapsed)

In [ ]:
from pathlib import Path

p = Path("./finetuned_models/")
filtered = [x.absolute() for x in p.glob("*.pt") if not x.name.endswith("best.pt")]

filtered[0].absolute()

PosixPath('/mnt/d/Projects/7007G/surgical-fine-tune/finetuned_models/10k-001-1.pt')

In [ ]:
dataloaders = dataloaders_dict
res = []

for weights_path in filtered:
    print(weights_path.stem, end='\t')
    model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')
    model.load_state_dict(torch.load(weights_path))
    dataloaders = dataloaders_dict
    e_res = list(eval_model(model,dataloaders))
    e_res.append(weights_path.stem)
    res.append(e_res)


5k-0001-3	epoch_loss=1.3252680839920044, epoch_acc_top1=0.6817200000000001, epoch_acc_top5=0.88438, time_elapsed=136.4023196697235
[[1.3252680839920044, 0.6817200000000001, 0.88438, 136.4023196697235, '5k-0001-3']]
5k-0001-4	epoch_loss=1.354621337966919, epoch_acc_top1=0.6859400000000001, epoch_acc_top5=0.88424, time_elapsed=136.43243217468262
[[1.3252680839920044, 0.6817200000000001, 0.88438, 136.4023196697235, '5k-0001-3'], [1.354621337966919, 0.6859400000000001, 0.88424, 136.43243217468262, '5k-0001-4']]
5k-001-1	epoch_loss=1.4621760076904298, epoch_acc_top1=0.6560800000000001, epoch_acc_top5=0.86754, time_elapsed=137.88075351715088
[[1.3252680839920044, 0.6817200000000001, 0.88438, 136.4023196697235, '5k-0001-3'], [1.354621337966919, 0.6859400000000001, 0.88424, 136.43243217468262, '5k-0001-4'], [1.4621760076904298, 0.6560800000000001, 0.86754, 137.88075351715088, '5k-001-1']]
5k-001-2	epoch_loss=1.4689690462493896, epoch_acc_top1=0.65332, epoch_acc_top5=0.8647, time_elapsed=138.29

In [ ]:
model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')
model.load_state_dict(torch.load('./10k-001-3.pt'))
dataloaders = dataloaders_dict
eval_model(model,dataloaders)

epoch_loss=1.3964715154266358, epoch_acc_top1=tensor(0.6766, device='cuda:0', dtype=torch.float64), epoch_acc_top5=0.8803, time_elapsed=113.3026909828186


In [ ]:
eval_model(model,dataloaders)

epoch_loss=1.4836121859741211, epoch_acc_top1=tensor(0.6525, device='cuda:0', dtype=torch.float64), epoch_acc_top5=0.8675, time_elapsed=113.71439123153687


In [ ]:
model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')
# model.load_state_dict(torch.load('./10k-001-3-100e.pt'))
dataloaders = dataloaders_dict
eval_model(model,dataloaders)

epoch_loss=0.9615748849105835, epoch_acc_top1=tensor(0.7614, device='cuda:0', dtype=torch.float64), epoch_acc_top5=0.92872, time_elapsed=115.07258129119873


In [ ]:
model

<All keys matched successfully>

In [ ]:
# results of IMAGENET1K_V1 on imagenet val
# epoch_loss, epoch_acc, time_elapsed/60
# epoch_loss = 0.96157
# epoch_acc = 0.7614
# time_elapsed 118.823552

0.9615749324417114